# Исследование надежности заемщиков

## Описание проекта

**Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.**

## Описание данных

**children** — количество детей в семье\
**days_employed** — общий трудовой стаж в днях\
**ob_years** — возраст клиента в годах\
**education** — уровень образования клиента\
**education_id** — идентификатор уровня образования\
**family_status** — семейное положение\
**family_status_id** — идентификатор семейного положения\
**gender** — пол клиента\
**income_type** — тип занятости\
**debt** — имел ли задолженность по возврату кредитов\
**total_income** — ежемесячный доход\
**purpose** — цель получения кредита**

## Откроем таблицу и изучим общую информацию о данных

**Импортируем библиотеку pandas и сохраним данные в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [2]:
import pandas as pd # импорт библиотеки pandas

In [3]:
data = pd.read_csv('/datasets/data.csv') # прочитали csv-файл

**Выведем первые 20 строчек датафрейма `data` на экран.**

In [4]:
print (data.head(20)) # ВывОд первых 20 строчек датафрейма data на экран

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

**Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [5]:
data.info() # информация о датафрейме 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Выведем количество пропущенных значений для каждого столбца, используя комбинацию двух методов.**

In [6]:
print(data.isna().sum()) # количество пропущенных значений для каждого столбца

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце мы обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [7]:
for profession in data['income_type'].unique(): # заполняем пропуски в столбце total_income медианой 
    data.loc[(data['income_type'] == profession)&(data['total_income'].isna()), 'total_income'] = \
    data.loc[data['income_type'] == profession, 'total_income'].median() #по каждому типу занятонсти 

### Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [8]:
data.loc[data['days_employed']<0, 'days_employed'] = \
data.loc[data['days_employed']<0, 'days_employed'].abs() # заменили все отрицательные значения положительными 


**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [14]:
print (data.groupby('income_type')['days_employed'].median()) # выводим медианное значение трудового стажа days_employed в днях

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64


У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

**Выведем перечень уникальных значений столбца `children`.**

In [28]:
print(data['children'].unique()) # Выводим перечень уникальных значений столбца children

[ 1  0  3  2 -1  4 20  5]


**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [54]:
#data = data[((data['children'] != -1) and (data['children'] != 20))] # ваш код здесь
#print(data['children'].unique())
data = data[data['children']!=-1] #Удаляем строки, в которых встречаются аномальные значения
data = data[data['children']!=20]


**Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [53]:
print(data['children'].unique()) # Выводим перечень уникальных значений столбца children

[1 0 3 2 4 5]


### Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.**

In [56]:
for profession in data['income_type'].unique(): # заполняем пропуски в столбце days_employed медианой 
    data.loc[(data['income_type'] == profession)&(data['days_employed'].isna()), 'days_employed'] = \
    data.loc[data['income_type'] == profession, 'days_employed'].median() #по каждому типу занятонсти 

**Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.**

In [58]:
print (data.isna().sum()) # Выводим количество пропущенных значений для каждого столбца с помощью двух методов

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [62]:
data['total_income'] = data['total_income'].astype('int')

### Обработка дубликатов

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведём их к нижнему регистру.**

In [64]:
data['education'] = data['education'].str.lower() # Приводим  к нижнему регистру.

**Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [66]:
print (data.duplicated().sum()) # считаем дубликаты

71


In [69]:
data = data.drop_duplicates()# удаляем дубликаты


0


### Категоризация данных

**На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 назначим категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [74]:
def categorize_income(row): # создаем функцию categorize_income(), которая разделит кредитополучателей на категории
    if 0<=row['total_income']<=30000:
        return ('E')
    elif 3001<=row['total_income']<=50000:
        return ('D')
    elif 5001<=row['total_income']<=200000:
        return ('C')
    elif 200001<=row['total_income']<=1000000:
        return ('B')
    elif 1000001<=row['total_income']:
        return ('A')



In [75]:
data['total_income_category'] = data.apply(categorize_income, axis=1) # применяем функцию методом apply()

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [77]:
print(data['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут нам правильно определить категорию.**

In [ ]:
purpose_car = ['приобретение автомобиля', 'на покупку подержанного автомобиля', \
                         'на покупку своего автомобиля', 'автомобили', \
                         'сделка с подержанным автомобилем', 'автомобиль',\
                         'свой автомобиль', 'сделка с автомобилем', 'на покупку автомобиля']
purpose_house = ['покупка жилья', 'операции с жильем', 'покупка жилья для семьи', \
                           'покупка недвижимости', 'покупка коммерческой недвижимости', \
                           'покупка жилой недвижимости', 'строительство собственной недвижимости', \
                           'недвижимость', 'строительство недвижимости', \
                           'операции с коммерческой недвижимостью', 'строительство жилой недвижимости', \
                           'жилье', 'операции со своей недвижимостью', 'покупка своего жилья', \
                           'операции с недвижимостью', 'покупка жилья для сдачи', 'ремонт жилью']
purpose_wedding = ['сыграть свадьбу', 'на проведение свадьбы', 'свадьба']
purpose_education = ['дополнительное образование', 'образование', 'заняться образованием', \
                     'получение образования', 'получение дополнительного образования', \
                     'получение высшего образования', 'профильное образование', 'высшее образование',\
                     'заняться высшим образованием']
def categorize_purpose(row): # создаем функцию categorize_purpose(), которая разделит кредитополучателей на категории
    if row['purpose'] in purpose_car:                  
        return ('операции с автомобилем')
    if row['purpose'] in purpose_house:
        return ('операции с недвижимостью')
    if row['purpose'] in purpose_wedding:
        return ('проведение свадьбы')
    if row['purpose'] in purpose_education:
        return ('получение образования')

  

In [ ]:
data['purpose_category'] = data.apply(categorize_purpose, axis=1) # применяем функцию методом apply()